In [ ]:
from pathlib import Path
tune_dir = Path("../models/ray").resolve()
tune_dir.exists()

In [ ]:
tunelogs = [d for d in tune_dir.iterdir()]
tunelogs.sort(reverse=True)
latest = tunelogs[0]
latest

In [ ]:
from ray.tune import ExperimentAnalysis
import ray
ray.init(ignore_reinit_error=True)


analysis = ExperimentAnalysis(latest)

In [ ]:
analysis.results_df.columns


In [ ]:
import plotly.express as px

plot = analysis.results_df
select = ["config/hidden", "config/dropout", "config/num_layers",'F1scoremicro',
       'F1scoremacro', 'Precisionmicro', 'Recallmacro','Accuracy']
p = plot[select].reset_index().dropna()
p

Let's sort by accuracy

In [ ]:
p.sort_values("Accuracy", inplace=True)

Make a parallel plot

In [ ]:
px.parallel_coordinates(p, color="Accuracy")


In [ ]:
import seaborn as sns

sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")

In [ ]:
import matplotlib.pyplot as plt
cmap = sns.cubehelix_palette(as_cmap=True)
sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")
sns.kdeplot(data=p, x="config/hidden_size", y="config/num_layers", cmap=cmap)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

Get the best trial

In [ ]:
analysis.get_best_trial(metric="test_loss", mode="min")


The top ten

In [ ]:
p[-10:]

Or the best config

In [ ]:
analysis.get_best_config(metric="Accuracy", mode="max")
